## Creating experiment groups

In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import lab
from lab.classes import database, ExperimentSet

### loading HR experiment groups

In [8]:
def loadHRExpGrp(expSet, mice, verbose=False, minLicks=5, minDuration=8, minLaps=0):
    """Loads hidden reward experiments and returns an experiment group dictionary

    expSet -- ExperimentSet object
    mice -- list of mice IDs, if one mouse then (["mouseID"]) if more than ("m1", "m1")
    """
    expGroups=dict();

    expGroup=lab.classes.HiddenRewardExperimentGroup.fromMice([expSet.grabMouse(mouse) for mouse in mice], label="hidden_behavior");
    expGroup.removeMissingBehaviorData();
    expGroup.removeMissingBehaviorData(key="treadmillPosition", verbose=verbose);
    expGroup.removeMissingBehaviorData(key="licking", verbose=verbose);
    expGroup.removeMissingBehaviorData(key="reward", verbose=verbose);
    expGroup.removeShortExperiments(remove_incomplete=True, laps=minLaps, verbose=verbose);
    expGroups["behavior"]=expGroup;

#     expGroups["behavior"].filterby(lambda df: ~df["condition"].str.contains("B"), ["condition"]);
#     expGroups["behavior"].filterby(lambda df: df["exposure"]<3, ["exposure"]);

    expGroups["imaging"]=lab.ExperimentGroup(expGroups["behavior"], label="hidden_imaging");
    expGroups["imaging"].removeDatalessExperiments(channel="Ch2", verbose=verbose);

    return expGroups;

In [21]:
mouse = loadHRExpGrp(expSet, (["gtm4_1"]))

In [72]:
for item in mouse["behavior"]:
    lick = item.keys

In [76]:
print(lick())

['trial_id', 'session', 'lap_tolerance', 'experimentType', 'laps', 'stopTime', 'uniqueLocationKey', 'belt', 'tSeriesDirectory', 'track_length', 'lap_reset_tag', 'position_scale', 'startTime', 'trial_length', 'sensors', 'day', 'condition', 'contexts', 'uses', 'controllers', 'sync_pin', 'imagingLayer', 'experimenter', 'reward', 'trial_attrs']


In [47]:
lickdf = pd.DataFrame(lick)
lickdf

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.003096,0.00,0.009288,0.00,0.00,0.00,0.00,0.00,0.00
1,0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.9,0.910000,0.92,0.930000,0.94,0.95,0.96,0.97,0.98,0.99


### loading HR experiment groups and properties

In [15]:
def loadHRExpsAndProps(expSet, mouseIDs, **kwargs):
    expAdd=dict();
    expAdd["mouseID"]=[];
    expAdd["expStartTime"]=[];
    expAdd["cds"]=[];  #cds = condition-day-session
    
    expAdd["expt"]=map(lambda mID, eST: expSet.grabExpt(mID, eST), expAdd["mouseID"], expAdd["expStartTime"]);
    expAdd=pd.DataFrame(expAdd);
    expAdd.drop(["mouseID", "expStartTime"], axis=1, inplace=True);
    expAdd.rename(columns={"cds": "condition_day_session"}, inplace=True);
    expAdd["condition_day"]=map(lambda cds: cds[0:3], expAdd["condition_day_session"]);
    expAdd["condition"]=map(lambda cds: cds[0], expAdd["condition_day_session"]);

    expGroups=loadHRExpGrp(expSet, mouseIDs, verbose=False, minLaps=3);

    expProps=lab.ExperimentGroup.dataframe(expGroups["imaging"],
                                           include_columns=["condition", "condition_day", "condition_day_session"]);

    expProps.drop(["mouse", "trial", "value"], axis=1, inplace=True);
    expProps=expProps.append(expAdd);
    expProps.drop_duplicates(subset=["expt"], keep="last", inplace=True);

    expGroups["imaging"]=list(expGroups["imaging"]);
    expGroups["imaging"].extend(list(expAdd["expt"]));
    expGroups["imaging"]=lab.ExperimentGroup(expGroups["imaging"], label="hidden_imaging");
    expGroups["imaging"].removeMissingBehaviorData(key="treadmillPosition");
    expGroups["imaging"].removeMissingBehaviorData(key="licking");
    expGroups["imaging"].removeMissingBehaviorData(key="reward");

    return expProps #, expGroups   

In [6]:
expSet = ExperimentSet("mossy.sql")

In [20]:
mossyMice = loadHRExpsAndProps(expSet, ("gtm4_1","gtm4_2"))

In [ ]:
mossyMice.